<a href="https://colab.research.google.com/github/IshanaySharma/Machine_learning_projects/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import numpy as np 

In [ ]:
#Tokenization
nltk.download('punkt') #downloading model to tokenize message
from nltk.tokenize import word_tokenize
#Stop words removal
nltk.download('stopwords') #downloading stopwords
from nltk.corpus import stopwords
#Lemmatization i.e. converting words into base form for eg. moving-> move
nltk.download('wordnet') #downloading all lemmas of english language
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#Vectorization
def clean_corpus(corpus):
    # lowering every word in text
    corpus = [ doc.lower() for doc in corpus]
    cleaned_corpus = []
    stop_words = stopwords.words('english')
    wordnet_lemmatizer = WordNetLemmatizer()
    # iterating over every text
    for doc in corpus:
        # tokenizing text
        tokens = word_tokenize(doc)
        cleaned_sentence = []
        for token in tokens:
            # removing stopwords, and punctuation
            if token not in stop_words and token.isalpha():
                # applying lemmatization
                cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token))
                cleaned_corpus.append(' '.join(cleaned_sentence))
    return cleaned_corpus

In [ ]:
import json
with open('/content/intents.json') as file:
  intents = json.load(file)

In [ ]:
corpus = []
tags = []
for intent in intents['intents']:
    # taking all patterns in intents to train a neural network
    for pattern in intent['patterns']:
        corpus.append(pattern)
        tags.append(intent['tag'])

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tags).reshape(-1,1))

In [ ]:
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential([Dense(128, input_shape=(X.shape[1],), activation='relu'), Dropout(0.2), Dense(64, activation='relu'), Dropout(0.2), Dense(y.shape[1], activation='softmax')])
model.compile(loss='categorical_crossentropy', optimizer='adam',
metrics=['accuracy'])
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1)

Epoch 1/20
88/88 [==============================] - 1s 2ms/step - loss: 2.8823 - accuracy: 0.0795
Epoch 2/20
88/88 [==============================] - 0s 2ms/step - loss: 2.7417 - accuracy: 0.1818
Epoch 3/20
88/88 [==============================] - 0s 2ms/step - loss: 2.5539 - accuracy: 0.2273
Epoch 4/20
88/88 [==============================] - 0s 2ms/step - loss: 2.2697 - accuracy: 0.3295
Epoch 5/20
88/88 [==============================] - 0s 2ms/step - loss: 1.8941 - accuracy: 0.6023
Epoch 6/20
88/88 [==============================] - 0s 2ms/step - loss: 1.5089 - accuracy: 0.6818
Epoch 7/20
88/88 [==============================] - 0s 2ms/step - loss: 1.1198 - accuracy: 0.8068
Epoch 8/20
88/88 [==============================] - 0s 2ms/step - loss: 0.7798 - accuracy: 0.9091
Epoch 9/20
88/88 [==============================] - 0s 2ms/step - loss: 0.5671 - accuracy: 0.9091
Epoch 10/20
88/88 [==============================] - 0s 3ms/step - loss: 0.4264 - accuracy: 0.9659
Epoch 11/20
88/88 [

In [ ]:
# if prediction for every tag is low, then we want to classify that message as noanswer
INTENT_NOT_FOUND_THRESHOLD = 0.40
def predict_intent_tag(message):
    message = clean_corpus([message])
    X_test = vectorizer.transform(message)
    y = model.predict(X_test.toarray())
    # if probability of all intent is low, classify it as noanswer
    if y.max() < INTENT_NOT_FOUND_THRESHOLD:
        return 'noanswer'
    prediction = np.zeros_like(y[0])
    prediction[y.argmax()] = 1
    tag = encoder.inverse_transform([prediction])[0][0]
    return tag
print(predict_intent_tag('How you could help me?'))
print(predict_intent_tag('swiggy chat bot'))
print(predict_intent_tag('Where is my order'))

IndexError: ignored

IndexError: ignored